# Draft analysis 

---

Group name: Doreen Mack, David Riethmann

---


## 1 Introduction

For our project we have chosen a dataset about the release and establishment of the weevil Mecinus janthiniformis for biological control of Dalmatian toadflax in southern California. Our motivation is to find patterns and trends within the data and define factors and potential predictors associated with the growth of Dalmatian toadflax and the spread of weevil populations.

Dalmatian toadflax is an invasive plant that is considered an undesirable weed in southern California. It can thrive and spread rapidly in a variety of ecosystems, including grasslands and roadsides. Invasive plants such as Dalmatian toadflax can cause ecological problems. On the on hand, it can displace native plants and as a result reduce biodiversity. It can also have an impact on the availability of habitat for native animals as the dominance of Dalmatian toadflax can disrupt the strucutre and function of ecosystems. On the other hand, we have to think of the soil composition. Invasive plants can alter the soil by removing nutrients which can have an impact on the native vegetation. Moreover, Dalmatian toadflax can tend to accumulate dry material, which can increase fire hazard.

Therefore, attempts are being made to control the spread of Dalmatian toadflax using weevil populations. They were released in the investigated area of southern California.

General research questions:

- Which aspects can help predict the total main stem length?
- Are plants that are infested by weevils really more likely to remain smaller?

Our hypothesis is that the spread of weevil populations really contributes to the reduction of Dalmatian toadflax vegetation in this area.

## 2 Setup

We start by importing the required libraries and modules.
These are:
- Numpy: A library for numerical operations which provides support for arrays and matrices along with mathematical functions to operate on these elements.
- Pandas: A library for data analysis which provides data structures such as series and DataFrames that form the basis for working with data.
- Altair: A library for statistical visualization that provides a variety of graphics.
- Scikit-Learn: A machine learning library which provides tools for data analysis and modelling. From Scikit-Learn we have imported a bunch of modules e.g. to calculate statistical measures or split the dataset.

In [1]:
import numpy as np
import pandas as pd
import altair as alt

from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report
# from sklearn.metrics import ConfusionMatrixDisplay
# from sklearn.metrics import RocCurveDisplay
# from sklearn.metrics import roc_auc_score

## 3 Data

### 3.1 Import data

Next we are using pandas to read the data form our CSV file and create a DataFrame which we call "df" and which will contain the data from the CSV file in tabular format. 

We also create a DataFrame "meta" which will contain our data dictionary. The data dictionary provides additional information about the columns and variables in "df". To read the data from our excel file we also use the pandas function.

In [2]:
df = pd.read_csv('../data/raw/dissections_2012_HV.csv')
meta = pd.read_excel('../references/Data Dictionary.xlsx')

### 3.2 Data structure

Let´s fist take a look at the metadata. We want to get a brief overview of the data in our "df".
We can see there are 25 variables. The description, role, type and format of each variable in our DataFrame can be taken from the following table:

In [3]:
meta

,Name,Description,Role,Type,Format
0,year,year that stems were infested,-,numeric,int64
1,diss date,date dissected in the laboratory,-,numeric,object
2,date,date collected in the field,-,numeric,object
3,site,six study sites at Hungry Valley study area,Predictor,nominal,object
4,trt,release or not in 2008 and 2014,-,nominal,object
5,BC,"1 = early establishment, 0 = late establishment",-,nominal,int64
6,stem #,stem ID,ID,numeric,int64
7,stem diam bottom (mm),diameter of stem at bottom,Predictor,numeric,float64
8,main stem length (cm),"length of stem, excluding side branches","Predictor, response",numeric,float64
9,total meja,"sum of no. empty chambers, dead larvae, dead p...",Predictor,numeric,int64


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1066 entries, 0 to 1065
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    1066 non-null   int64  
 1   diss date               1066 non-null   object 
 2   Date                    1066 non-null   object 
 3   site                    1066 non-null   object 
 4   trt                     1066 non-null   object 
 5   BC                      1066 non-null   int64  
 6   stem #                  1066 non-null   int64  
 7   stem diam bottom (mm)   1066 non-null   float64
 8    main stem length (cm)  1066 non-null   float64
 9   Total Meja              1066 non-null   int64  
 10  Meja/100 cm             1066 non-null   float64
 11  infested                1066 non-null   int64  
 12  diam top (mm)           370 non-null    float64
 13  Tip of Stem broken      500 non-null    object 
 14  side branches (cm)      84 non-null     

In [5]:
df.head().T

,0,1,2,3,4
year,2012,2012,2012,2012,2012
diss date,12/7/12,12/8/12,12/10/12,12/10/12,12/10/12
Date,12/3/12,12/3/12,12/3/12,12/3/12,12/3/12
site,West,West,West,West,West
trt,Release,Release,Release,Release,Release
BC,1,1,1,1,1
stem #,1,2,3,4,5
stem diam bottom (mm),7.8,8.2,6.6,6.4,6.5
main stem length (cm),64.0,57.5,33.0,59.0,45.0
Total Meja,21,46,32,30,31


In [6]:
df.tail().T

,1061,1062,1063,1064,1065
year,2018,2018,2018,2018,2018
diss date,8/23/19,8/23/19,8/23/19,8/23/19,8/23/19
Date,5/9/19,5/9/19,5/9/19,5/9/19,5/9/19
site,USFS-North,USFS-North,USFS-North,USFS-North,USFS-North
trt,Check,Check,Check,Check,Check
BC,1,1,1,1,1
stem #,16,17,18,19,20
stem diam bottom (mm),4.5,5.3,6.8,6.9,7.1
main stem length (cm),46.0,51.0,109.0,104.0,96.0
Total Meja,8,1,4,1,2


We use the describe function to compute summary statistics for all numerical columns in our DataFrame. It includes the count, mean, standard deviation, minimum, 25th percentile (Q1), 50th percentile (median), 75th percentile (Q3), and maximum. 
To switch rows and columns the transpose operation is used.

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
year,1066.0,2014.288931,2.034678,2012.0,2012.000,2014.00,2016.000,2018.0
BC,1066.0,0.487805,0.500086,0.0,0.000,0.00,1.000,1.0
stem #,1066.0,25.863039,29.670016,1.0,8.000,15.00,32.000,141.0
stem diam bottom (mm),1066.0,4.963227,1.832666,1.1,3.700,4.90,6.200,12.2
main stem length (cm),1066.0,51.277674,24.923005,2.0,34.000,49.00,67.000,135.0
Total Meja,1066.0,5.273921,9.163923,0.0,0.000,0.00,8.000,67.0
Meja/100 cm,1066.0,11.513602,20.270924,0.0,0.000,0.00,15.550,133.3
infested,1066.0,0.454034,0.498116,0.0,0.000,0.00,1.000,1.0
diam top (mm),370.0,1.781081,1.224510,0.2,0.900,1.50,2.300,7.6
side branches (cm),84.0,60.226190,47.528647,8.0,29.500,45.50,76.750,250.0


### 3.3 Data corrections

Data corrections must be carried out as a basis for data analysis. We start by taking a look at the columns designations:

In [8]:
df.columns

Index(['year', 'diss date', 'Date', 'site', 'trt', 'BC', 'stem #',
       'stem diam bottom (mm)', ' main stem length (cm)', 'Total Meja',
       'Meja/100 cm', 'infested', 'diam top (mm)', 'Tip of Stem broken',
       'side branches (cm)', 'No. empty chambers', 'total chamber length',
       'live adults', 'dead adults', 'dead larvae', 'dead pupae', 'parasitoid',
       'live pupa', 'live larva', 'Total No. adults'],
      dtype='object')

We quickly see that the column names have some peculiarities that can lead to problems for example upper and lower case, special characters. Therefore we want to standardize column names and make them more consistent by using regular expressions.

We start by using str.lower in order to convert all characters in string to lowercase.

In [9]:
df.columns = df.columns.str.lower()

For the column "stem #" we want to replace # by ID.

In [10]:
df.columns = df.columns.str.replace(r"\#", r"id", regex=True)

Next we are removing leading and trailing whitespaces from each column name.

In [11]:
df.columns = df.columns.str.replace(r"^\s+|\s+$", r"", regex=True)

Moreover we are replacing all spaces in each column name with underscores.

In [12]:
df.columns = df.columns.str.replace(r"\s", r"_", regex=True)

In the beginning we saw that many special characters are being used and units are given in brackets. With the next codeline we are replacing all opening brackets in each column name with "in_".

In [13]:
df.columns = df.columns.str.replace(r"\(", r"in_", regex=True)

After that we are removing all closing brackets in each column name.

In [14]:
df.columns = df.columns.str.replace(r"\)", r"", regex=True)

Moreover we have to remove all periods and replace all forward slashes in each column name with "per".

In [15]:
df.columns = df.columns.str.replace(r"\/", r"_per_", regex=True)

In [16]:
df.columns = df.columns.str.replace(r"\.", r"", regex=True)

In the next codelines we are replacing no by number and year by year_infested.

In [17]:
df.columns = df.columns.str.replace(r"no", r"number", regex=True)

In [18]:
df.rename(columns={'year': 'year_infested'}, inplace=True)

As some columns contain no data or data that is not relevant for further analysis, we omit the corresponding columns.

In [19]:
df = df.drop(columns=['bc', 'trt', 'live_adults', 'dead_adults', 'dead_larvae', 'dead_pupae', 'parasitoid', 'live_pupa', 'live_larva'])

In the "tip_of_stem_broken" column, c stands for cut and b for broken. If neither c nor b is given, we assume that the tip of the stem is intact. An intact tip leads to a greater length of the main stem, and we want to use the data as a categorical predictor. Therefore, a broken or cut stem tip is assigned the value 1 and an intact stem tip is assigned the value 0. We are using a loop to iterate over the values in the 'tip_of_stem_broken' column. For each value, it checks if the value is equal to 'b' or 'c'. If the condition is true, it assigns the value 1 to the corresponding cell in the 'tip_of_stem_broken' column, otherwise, it assigns the value 0.

In [20]:
for v in range(len(df['tip_of_stem_broken'])):
    if df.loc[v, 'tip_of_stem_broken'] == 'b' or df.at[v, 'tip_of_stem_broken'] == 'c':
        df.loc[v, 'tip_of_stem_broken'] = 1
    else:
        df.loc[v, 'tip_of_stem_broken'] = 0

Afterwards we converte the values in the column to integer type.

In [21]:
df['tip_of_stem_broken'] = df['tip_of_stem_broken'].astype('int64')

In [22]:
#df.dropna(axis=1, inplace=True)

Moreover a few other convertions have to be made.
The values in column "site" are converted to category type and the values in columns "date" and "diss_date" are converted to datetime objects but we only extract the date component.

In [23]:
df['site'] = df['site'].astype('category')

In [24]:
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')
df['date'] = df['date'].dt.date

In [25]:
df['diss_date'] = pd.to_datetime(df['diss_date'], format='%m/%d/%y')
df['diss_date'] = df['diss_date'].dt.date

At last we are converting the values in the two diameter columns from millimeters to centimeters and round to two decimal places. New columns are crated for the diameter columns in centimeters.

In [26]:
df['stem_diam_bottom_in_cm'] = df.loc[:, 'stem_diam_bottom_in_mm'] / 10

In [27]:
df[['stem_diam_bottom_in_cm']] = df[['stem_diam_bottom_in_cm']].round(2)

In [28]:
df['diam_top_in_cm'] = df.loc[:, 'diam_top_in_mm'] / 10

This is the final result of the columns names:

In [29]:
df.columns

Index(['year_infested', 'diss_date', 'date', 'site', 'stem_id',
       'stem_diam_bottom_in_mm', 'main_stem_length_in_cm', 'total_meja',
       'meja_per_100_cm', 'infested', 'diam_top_in_mm', 'tip_of_stem_broken',
       'side_branches_in_cm', 'number_empty_chambers', 'total_chamber_length',
       'total_number_adults', 'stem_diam_bottom_in_cm', 'diam_top_in_cm'],
      dtype='object')

Finally we save the corrected DataFrame as CSV file.

In [30]:
df.to_csv('../data/interim/dissections_2012_HV_corrected.csv')

### 3.4 Variable lists

To get an overview of the variables we now have in our corrected DataFrame, we call df.info(). This method provides a summary of the DataFrame, including information about its structure, e.g. the number of entries and the number of non-null values per column, as well as its data types.

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1066 entries, 0 to 1065
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   year_infested           1066 non-null   int64   
 1   diss_date               1066 non-null   object  
 2   date                    1066 non-null   object  
 3   site                    1066 non-null   category
 4   stem_id                 1066 non-null   int64   
 5   stem_diam_bottom_in_mm  1066 non-null   float64 
 6   main_stem_length_in_cm  1066 non-null   float64 
 7   total_meja              1066 non-null   int64   
 8   meja_per_100_cm         1066 non-null   float64 
 9   infested                1066 non-null   int64   
 10  diam_top_in_mm          370 non-null    float64 
 11  tip_of_stem_broken      1066 non-null   int64   
 12  side_branches_in_cm     84 non-null     float64 
 13  number_empty_chambers   1066 non-null   int64   
 14  total_chamber_length    

#### 3.4.1 Linear Regression

Now we need to prepare our linear regression analysis. As y_label we define the main stem length in cm, as this is the dependent variable we want to predict. Next, we define our possible predictor variables in a list called features:

- stem_diam_bottom_in_cm
- total_meja
- meja_per_100_cm
- diam_top_in_cm
- side_branches_in_cm

We want to use these variables to predict the main stem length of the plants and therefore define them as X.
Y is assigned a series containing the values from the column main stem length that is defined by the y_label variable.

In [32]:
y_label = 'main_stem_length_in_cm'

features = ['stem_diam_bottom_in_cm', 'total_meja', 'meja_per_100_cm', 'diam_top_in_cm', 'side_branches_in_cm'] #, 'tip_of_stem_broken', 'infested', 'meja_per_100_cm', 'total_number_adults'

#### 3.4.2 Logistic Regression

For the logistic regression analysis, we want to predict whether a plant is infested or not in order to analyze whether plants infested by weevils are more likely to remain smaller and if weevils are really useful and effective as natural control agents.
Possible predictor variables are again defined in a features list:

- main_stem_length_cm
- stem_diam_bottom_cm
- side_branches_cm
- diam_top_cm
- tip_of_stem_broken
- site
- total_meja
- meja_per_100_cm
- total_number_adults

We want to use these variables to predict wheter the plant is infested or not and therefore define them as X.

In [33]:
# y_label_log = 'infested'

# features_log = ['main_stem_length_cm', 'stem_diam_bottom_cm', 'side_branches_cm', 'diam_top_in_cm', 'tip_of_stem_broken'] #, 'site', 'total_meja', 'meja_per_100_cm', 'total_number_adults'

### 3.5 Data splitting and imputation

Using df.info() method, we quickly recognized that columns diam_top_in_cm and side_branches_in_cm only contain fewer values.
We therefore have to fill in the missing values using SimpleImputer class from Scikit-Learn.
The following code lines are performing imputation on these missing values in the two specified columns. As strategy we specify median so the missing values are replaced with the median of each column. Moreover the imputed values are rounded to two decimal places 

In [34]:
# Imputer for filling missing values with the median
imputer = SimpleImputer(strategy='median')

# Imputing missing values in the predictors
data_imputed = df.copy()
data_imputed[['diam_top_in_cm', 'side_branches_in_cm']] = imputer.fit_transform(df[['diam_top_in_cm', 'side_branches_in_cm']])
data_imputed[['diam_top_in_cm']] = data_imputed[['diam_top_in_cm']].round(2)
data_imputed[['side_branches_in_cm']] = data_imputed[['side_branches_in_cm']].round(2)

# Note: We're only excluding rows with missing target values, as imputation has handled missing predictors
X_imputed = data_imputed.dropna(subset=[y_label])[features] 
y_imputed = data_imputed.dropna(subset=[y_label])[y_label] 

# X_imputed_log = data_imputed.dropna(subset=[y_label_log])[features_log] 
# y_imputed_log = data_imputed.dropna(subset=[y_label_log])[y_label_log] 

To continue, our data must be split into training and test sets. We use the train_test_split function from scikit-learn to perform the split. We define 20% of the data set to be included in the test split. The remaining 80% of the data set is therefore used as the training set.

Linear Regression:

In [35]:
X_train_imputed, X_test_imputed, y_train_imputed, y_test_imputed = train_test_split(
    X_imputed,
    y_imputed,
    test_size=0.2,
    random_state=0
)

After the splitting we create a new DatFrame called df_train from the training set and save it as CSV file.

In [36]:
df_train = pd.DataFrame(X_train_imputed.copy())
df_train = df_train.join(pd.DataFrame(y_train_imputed))
df_train.to_csv('../data/interim/train_data.csv')

Logistic Regression:

In [ ]:
X_train_imputed_log, X_test_imputed_log, y_train_imputed_log, y_test_imputed_log = train_test_split(
    X_imputed_log,
    y_imputed_log,
    test_size=0.2,
    random_state=0
)

In [ ]:
df_train_log = pd.DataFrame(X_train_imputed_log.copy())
df_train_log = df_train_log.join(pd.DataFrame(y_train_imputed_log))
df_train_log.to_csv('../data/interim/train_data_log.csv')

## 4 Analysis

### 4.1 Descriptive statistics

Typical methods of descriptive data analysis are the calculation of mean values, medians and standard deviations as well as the creation of histograms, box plots or cross tables. It focuses on the organization and description of existing data without in-depth analysis or hypothesis formation.

Again, we use the describe function to compute summary statistics for all numerical columns in our corrected Test-DataFrame.

In [37]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
stem_diam_bottom_in_cm,852.0,0.494859,0.183730,0.11,0.360,0.48,0.62,1.12
total_meja,852.0,5.252347,8.949443,0.00,0.000,0.00,8.00,62.00
meja_per_100_cm,852.0,11.508333,20.038197,0.00,0.000,0.00,16.10,133.30
diam_top_in_cm,852.0,0.158732,0.074982,0.02,0.150,0.15,0.15,0.76
side_branches_in_cm,852.0,46.628521,13.425784,8.00,45.500,45.50,45.50,222.00
main_stem_length_in_cm,852.0,51.234155,24.998356,2.00,33.875,49.00,67.00,135.00


Next, we create a grid of bar charts using Altair. Each column represents a different variable, either a feature or the response variable. Each row represents a histogram of the variables distribution.
This diagram is a tool for an initial, visual exploration of the data distributions in the training dataset. The histogram grids allow us to check the distribution of each variable in the training data set. We can identify patterns, outliers or characteristic shapes in the histograms that may indicate certain properties of the data.

In [38]:
alt.Chart(df_train).mark_bar().encode(
    alt.X(alt.repeat("column"), type="quantitative", bin=True),
    y='count()',
).properties(
    width=150,
    height=150
).repeat(
    column=[y_label] + features
)

alt.RepeatChart(...)

### 4.2 Exploratory data analysis

EDA uses a variety of methods to discover patterns in the data. These include scatterplots, correlation analysis, clustering and other exploratory visualizations. The focus is on gaining new insights and understanding the data set.

First, we create a grid matrix of scatter plots. This matrix provides a visual overview of the relationship between different pairs of variables in the training dataset. By looking at the scatter plots, patterns such as linear relationships, clustering or outliers can be identified.
The repetition in both the rows and columns makes it possible to look at relationships between the target variable (y_label) and the other features, as well as between the features themselves.

In [39]:
alt.Chart(df_train).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative')
).properties(
    width=150,
    height=150
).repeat(
    row=[y_label] + features,
    column=[y_label] + features
).interactive()

alt.RepeatChart(...)

We can see that stem diam bottom and main stem length have a linear relationship.

In the following code cell we calculate the correlation coefficients between the columns in the DataFrame df_train and the target variable (y_label).
The higher the correlation coefficient, the stronger the linear relationship between the two variables. A positive correlation indicates that as one variable increases, the other tends to increase, while a negative correlation indicates that as one variable increases, the other tends to decrease.

As we already recognized in the scatter plot matrix, stem diam bottom has the strongest linear relationship (0.66) with the response variable main stem length. This indicates that stem diam bottom will be a suitable predictor variable.
However, we have to keep in mind that correlation does not imply causation.

In [40]:
corr = df_train.corr()
corr[y_label].sort_values(ascending=False)

main_stem_length_in_cm    1.000000
stem_diam_bottom_in_cm    0.655359
total_meja                0.031019
side_branches_in_cm      -0.048774
diam_top_in_cm           -0.065324
meja_per_100_cm          -0.128625
Name: main_stem_length_in_cm, dtype: float64

By applying the background gradient, we can improve the visualization of the correlation matrix. Cells with higher positive correlations are shaded with darker blue colors, while cells with higher negative correlations are shaded with lighter blue colors. 

This color gradient allows for a quick and intuitive visual assessment of the strength and direction of correlations in the matrix.

In [41]:
corr.style.background_gradient(cmap='Blues')

,stem_diam_bottom_in_cm,total_meja,meja_per_100_cm,diam_top_in_cm,side_branches_in_cm,main_stem_length_in_cm
stem_diam_bottom_in_cm,1.000000,0.184213,0.079548,0.271085,0.086061,0.655359
total_meja,0.184213,1.000000,0.913216,0.140567,-0.040964,0.031019
meja_per_100_cm,0.079548,0.913216,1.000000,0.146412,-0.040138,-0.128625
diam_top_in_cm,0.271085,0.140567,0.146412,1.000000,-0.009801,-0.065324
side_branches_in_cm,0.086061,-0.040964,-0.040138,-0.009801,1.000000,-0.048774
main_stem_length_in_cm,0.655359,0.031019,-0.128625,-0.065324,-0.048774,1.000000


We can also see that total meja and meja per 100cm do have a high correlation. The value of 0.91 suggests a strong positive linear relationship. As meja per 100 cm is defined as total meja / main stem length (cm) * 100 (cm) this is not really surprising.

However, we need to be careful as high correlations between the predictor variables can lead to multicollinearity, which can affect the stability of the regression coefficients. As a solution, we can remove these highly correlated variables from the model.

Boxplot (infested/not infested) abhängig von main stem length

### 4.3 Relationships

In [42]:
# Evtl. oben bei EDA mit integrieren? Ist doch dasselbe?

## 5 Model

### 5.1 Select model

In the following code cells we create the LinearRegression object (reg) and the logistic regression object (clf).

In [43]:
reg = LinearRegression()

In [44]:
# clf = LogisticRegressionCV()

### 5.2 Training and validation

Next we are using cross-validation to evaluate the performance of our linear regression and logistic regression model on the training dataset.
This approach provides a robust estimate of the model's performance by evaluating it on five different subsets of the training data.

#### 5.2.1 Linear Regression

In [45]:
scores = cross_val_score(reg, X_train_imputed, y_train_imputed, cv=5, scoring='neg_mean_squared_error', error_score='raise') *-1

Now we are storing the cross-validation scores in a DataFrame called df_scores and then modifying its index and applying a background gradient style for visual representation. Darker shades of blue correspond to higher cross-validation scores. Column "lr" now contains the cross-validation scores for each fold.

In [46]:
df_scores = pd.DataFrame({"lr": scores})

df_scores.index += 1

df_scores.style.background_gradient(cmap='Blues')

,lr
1,286.724120
2,336.584012
3,199.579851
4,283.827043
5,288.641271


The results represent the negative mean squared errors for each of the five folds in the k-fold cross-validation for the linear regression model. The result indicates that the linear regression model makes relatively good predictions on our training dataset.

Next, we want to create a line chart to visualize the mean squared error (MSE) across the different folds. Each point on the line represents the mean squared error for a specific fold.
This visualization provides a way to assess how the model's performance varies across different folds and whether there are any noticeable patterns or trends.

In [47]:
alt.Chart(df_scores.reset_index()).mark_line(
     point=alt.OverlayMarkDef()
).encode(
    x=alt.X("index", bin=False, title="Fold", axis=alt.Axis(tickCount=5)),
    y=alt.Y("lr", aggregate="mean", title="Mean squared error (MSE)")
)

alt.Chart(...)

Using the describe method again we can create summary statistics for each variable, in this case, each fold represented by the "lr" column.

In [48]:
df_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
lr,5.0,279.071259,49.495883,199.579851,283.827043,286.72412,288.641271,336.584012


The table provides a summary of the distribution of mean squared error values across the different folds in our cross-validation. The relatively small standard deviation suggests that the mean squared error values are close to the mean, indicating consistency in model performance across the folds.

#### 5.2.2 Logistic Regression

In [49]:
# scores_clf = cross_val_score(clf, X, y, cv=5, scoring='accuracy') wir können hier auch was anderes als accuracy nehmen

Again we are storing the cross-validation scores for our logistic regression model in a DataFrame called df_scores_clf and then modifying its index and applying a background gradient style for visual representation.

In [50]:
# df_scores_clf = pd.DataFrame({"lr": scores_clf})

# df_scores_clf.index += 1

# df_scores_clf.style.background_gradient(cmap='Blues')

The table indicates that...

Using the describe method again we can create summary statistics for each fold represented by the "lr" column.

In [51]:
# df_scores_clf.describe().T

The table provides a summary of the distribution of accuracy values across the different folds in our cross-validation.

Ergebnis: 

### 5.3 Fit model

#### 5.3.1 Linear Regression

To fit or train our linear regression model to training data we are using the fit method.
It performs the training process in which the model learns the relationship between the independent and dependent variables in the training data. For linear regression, this means finding the optimal coefficients/weights to create a linear function that best represents the training data.

After fitting, the reg object is now a trained linear regression model that we can use to make predictions for new data that was not part of the training dataset.

In [52]:
reg.fit(X_train_imputed, y_train_imputed)

LinearRegression()

Now we create a table with the estimated coefficients for the intercept and the slopes of the independent variables in the trained linear regression model.The coefficients indicate how much the dependent variable changes when the corresponding independent variable changes by one unit.

In [53]:
intercept = pd.DataFrame({
    "Name": ["Intercept"],
    "Coefficient":[reg.intercept_]}
    )

slope = pd.DataFrame({
    "Name": features,
    "Coefficient": reg.coef_}
)

table = pd.concat([intercept, slope], ignore_index=True, sort=False)

round(table, 3)

,Name,Coefficient
0,Intercept,28.537
1,stem_diam_bottom_in_cm,94.222
2,total_meja,1.217
3,meja_per_100_cm,-0.688
4,diam_top_in_cm,-78.221
5,side_branches_in_cm,-0.214


Interpretation of the results:

- The intercept is approximately 28.537. This is the estimated value of the dependent variable when all independent variables are zero. 
- An increase of one unit in the bottom stem diameter size is associated with an increase of about 94.222 units in the dependent variable main stem length.
- A one unit increase in total meja is associated with an increase of approximately 1,217 units in the dependent variable main stem length.
- An increase of one unit in the number of meja per 100 cm is associated with a decrease of about 0.688 units in the dependent variable main stem length.
- A one unit increase in the top stem diameter size is associated with a decrease of about 78.221 units in the dependent variable main stem length.
- An increase of one unit in side branch length is associated with a decrease of approximately 0.214 units in the dependent variable main stem length.

#### 5.3.2 Logistic Regression

To fit or train our logistic regression model to training data we are again using the fit method.

In [54]:
#clf.fit(X_train_imputed, y_train_imputed)

Again we create a table with the estimated coefficients for the intercept and the slopes of the independent variables in the trained logistic regression model.

In [55]:
intercept = pd.DataFrame({
    "Name": ["Intercept"],
    "Coefficient":[clf.intercept_]}
    )

slope = pd.DataFrame({
    "Name": features,
    "Coefficient": clf.coef_}
)

table = pd.concat([intercept, slope], ignore_index=True, sort=False)

round(table, 3)

NameError: name 'clf' is not defined

### 5.4 Evaluation on test set

#### 5.4.1 Linear Regression Model

Now we use the previously trained linear regression model to generate predictions for the dependent variable based on our test data (X_test_imputed). These predictions can then be compared with the actual values (y_test_imputed) to evaluate the performance of the model.

In [ ]:
y_pred_imputed = reg.predict(X_test_imputed)

To evaluate our model, we use various functions from the sklearn.metrics module.

First, we use the r2_score function to calculate R² between the actual values and the predicted values.

In [ ]:
r2_score(y_test_imputed, y_pred_imputed).round(3)

The R² value is between 0 and 1, where 1 means that the model explains all variation in the dependent variables, and 0 means that the model has no explanatory power beyond predicting the average. An R² value of 0.605 shows that about 60.5% of the variations in the actual values can be explained by the model. This indicates that the model has moderate success in explaining the observed variations. However, there is still room for improvement as more than 39% of the variations are not explained by the model.

Next we calculate the mean squared error (MSE) between the actual values and the predicted values.

In [ ]:
mean_squared_error(y_test_imputed, y_pred_imputed).round(3)

The mean square error indicates the average size of the square deviations between the actual and predicted values. A lower MSE indicates more accurate predictions. 
In our case, a mean square error (MSE) of 239.282 indicates that the mean square error between the actual and predicted values of main stem length is about 239.282 square centimeters.

Now we want to calculate the square root of the root mean square error (RMSE).

In [ ]:
mean_squared_error(y_test_imputed, y_pred_imputed, squared=False).round(3)

The RSME indicates the average size of the absolute deviations between the actual and predicted values. A lower RMSE indicates more accurate predictions. In our case, the RMSE of 15.469 cm means that the average deviation between the actual and predicted values is about 15.469 cm.

We now that the minimum main stem length is 2 cm, the maximum main stem length is 135 cm and the standard deviation 24.92 cm.
In view of that the RMSE of 15.469 cm can be considered acceptable given the span of the main stem lengths.

At last we want to calulate the mean absolute error (MAE) between the actual values and the predicted values.

In [ ]:
mean_absolute_error(y_test_imputed, y_pred_imputed).round(3)

The MAE indicates the average size of the absolute deviations between the actual and predicted values. In your case, the MAE of 12.401 means that the average absolute deviation between the actual and predicted lengths is about 12.401 cm.

#### 5.4.2 Logistic Regression Model

In the following we use the trained logistic regression model to generate predictions for the dependent variable based on our test data (X_test_imputed).

The choice of threshold for binary classification problems is an important decision and can have a significant impact on the performance of the model. Since we need to categorize our predictions into infested or not infested, we set a threshold of 0.5.

In [ ]:
# y_pred_imputed_clf = clf.predict(X_test_imputed)

In [ ]:
# ConfusionMatrixDisplay.from_estimator(clf, X_test_imputed, y_test_imputed)

In [ ]:
# print(classification_report(y_test_imputed, y_pred_imputed_clf, target_names=['Not infested', 'Infested']))

Now we want to interpret precision, recall and f1 score of our logistic regression model:

**Precision:** 
Precision is the ratio of correctly predicted positive classes to all items predicted as positive. Therefore it tells us how correct or precise the positive predictions of our model are.

**Recall:**
Recall measures how many of the actual positive cases we were able to correctly predict. It is important if we believe that false negatives are more important than false positives.

**F1-Score:**
F1-Score is a performance metric that takes into account both precision and recall. It is calculated from the harmonic mean of the two metrics.


To further evaluate our logistic regression model, we can visualize the ROC curve (Receiver Operating Characteristic Curve). Previously, we had set the threshold value to 0.5. The ROC curve provides a visual way to observe changes in our model's classification thresholds that affect the model's performance. We plot the rate of true positives against the rate of false positives for all possible thresholds. This allows us to select the classification threshold at which we identify the most true-positive cases. 

In [ ]:
# RocCurveDisplay.from_estimator(clf, X_test_imputed, y_test_imputed)

The curve indicates that...


Next, we can calculate the Area Under the ROC Curve (AUC). It is a performance measure which is independent of the exact classification threshold chosen. AUC is between 0 and 1, with higher numbers indicating better performance.

In [ ]:
# y_score = clf.predict_proba(X_test_imputed)[:, 1]
# roc_auc_score(y_test_imputed, y_score)

The value of xx indicates that...

### 5.5 Save model



Save your model in the folder `models/`. Use a meaningful name and a timestamp.

## 6 Conclusions